##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción a Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

## Descripción general

Keras Tuner es una biblioteca que le ayuda a seleccionar el mejor conjunto de hiperparámetros para su programa de TensorFlow. El proceso de selección del conjunto adecuado de hiperparámetros para su aplicación de aprendizaje automático (ML, por sus siglas en inglés) se denomina *ajuste de hiperparámetros* o *hiperajuste*.

Los hiperparámetros son las variables que controlan el proceso de entrenamiento y la topología del modelo de aprendizaje automático. Dichas variables se mantienen constantes a lo largo del proceso de entrenamiento y afectan directamente el rendimiento del programa de aprendizaje automático. Existen dos tipos de hiperparámetros:

1. **Hiperparámetros de modelo** que influyen en la selección del modelo, como la cantidad y el ancho de las capas ocultas.
2. **Hiperparámetros de algoritmo** que influyen en la velocidad y la calidad del algoritmo de aprendizaje, como la tasa de aprendizaje del Descenso del Gradiente Estocástico (SGD, Stochastic Gradient Descent) y el número de vecinos más cercanos de un clasificador k vecinos más cercanos (KNN, K Nearest Neighbors)

En este tutorial, usará Keras Tuner para ejecutar el ajuste de hiperaparámetros en una aplicación de clasificación de imágenes.

## Preparar

In [ ]:
import tensorflow as tf
from tensorflow import keras

Instalar e importar Keras Tuner.

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

## Cómo descargar y preparar el conjunto de datos

En este tutorial, usará Keras Tuner para encontrar los mejores hiperparámetros para un modelo de aprendizaje automático que clasifica imágenes de prendas de vestir del conjunto de datos [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist).

Cargue los datos.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Definir el modelo

Cuando se desarrolla un modelo de hiperajuste, además de la arquitectura del modelo, es necesario definir el espacio de búsqueda del hiperparámetro. El modelo que se configura para el hiperajuste se denomina *hipermodelo*.

Hay dos formas de definir un hipermodelo:

- Usando una función de generador de modelos
- Subclasificando la clase `HyperModel` de la API de Keras Tuner

También se pueden usar dos clases predefinidas de [HyperModel](https://keras.io/api/keras_tuner/hypermodels/); [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) y [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) para aplicaciones de visión artificial.

En este tutorial, usaremos una función de generador de modelos para definir el modelo de clasificación de imágenes. La función de generador de modelos devuelve un modelo compilado y usa los hiperparámetros que usted define en línea para ejecutar el hiperajuste del modelo.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Crear una instancia del optimizador y ejecutar hiperajuste

Cree una instancia del optimizador para ejecutar el hiperajuste. Keras Tuner tiene cuatro optimizadores disponibles: `RandomSearch`, `Hyperband`, `BayesianOptimization`, y `Sklearn`. En este tutorial, usaremos el optimizador [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Para crear la instancia del optimizador Hyperband, debe especificar el hipermodelo, el `objective` que desea <br>optimizar y la cantidad máxima de épocas que desea entrenar (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

El algoritmo de ajuste Hyperband usa asignación adaptativa de recursos y detención temprana para converger rápidamente en un modelo de alto rendimiento. Esto se logra mediante la implementación de una llave del estilo de las competencias deportivas. El algoritmo entrena un gran número de modelos durante algunas épocas y solo avanza a la siguiente ronda con la mitad de los modelos que muestran un mejor rendimiento. Hyperband determina la cantidad de modelos que se entrenarán en una llave calculando 1 + log<sub><code>factor</code></sub>(`max_epochs`) y redondeándolo al entero más cercano.

Cree una retrollamada para detener el entrenamiento de forma temprana una vez que se haya alcanzado un valor específico en la pérdida de validación.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Ejecute la búsqueda de hiperparámetros. Los argumentos del método de búsqueda son los mismos que se usan para `tf.keras.model.fit` además de la retrollamada mencionada anteriormente.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

## Entrenar el modelo

Use los hiperparámetros obtenidos en la búsqueda para encontrar la cantidad óptima de épocas para entrenar el modelo.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Vuelva a crear instancias del hipermodelo y entrénelo con la cantidad óptima de épocas obtenidas en el paso anterior.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Para completar este tutorial, evalúe el hipermodelo en los datos de prueba.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

El directorio `my_dir/intro_to_kt` contiene registros detallados y puntos de verificación para cada prueba (configuración de modelo) que se ejecuta durante la búsqueda de hiperparámetros. Si vuelve a ejecutar la búsqueda de hiperparámetros, Keras Tuner usa el estado existente de esos registros para reanudar la búsqueda. Para inhabilitar este comportamiento, debe ingresar un argumento `overwrite=True` adicional durante la creación de la instancia del optimizador.

## Resumen

En este tutorial, aprendimos cómo usar Keras Tuner para ajustar los hiperparámetros de un modelo. Para obtener más información sobre cómo usar Keras Tuner, consulte los siguientes recursos adicionales:

- [Keras Tuner en el blog de TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Sitio web de Keras Tuner](https://keras-team.github.io/keras-tuner/)

También le recomendamos que consulte el [panel de control de HParams](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) en TensorBoard para ajustar los hiperparámetros de su modelo de forma interactiva.